rules of the game : https://cdn.1j1ju.com/medias/85/5e/7c-pickomino-rulebook.pdf

subject : https://team.inria.fr/polaris/files/2023/10/project_2023.pdf

#PART 1: MDP for 1 player

\go faire un truc déguelasse avec dico et sets et ensuite on optimise si besoin

on considère dans toute la suite que 6 est synonyme de 'W' (worm)

a state is : a sorted list of dices rolled + a set of faces you cannot choose + the sum of dices already picked

In [2]:
#Première question
from dice_state import *
from mdp2 import *

c = 0 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP2(c,r)

dices = (1, 3, 3, 3, 3, 4, 4, 5, 6)
score = 0
used = set()
state = DiceState(dices, score, used)

mdp.explore(state)

print(mdp.opti[state], "value is:", mdp.value[state])


Continue with dice 4 value is: 1.947539880511477


In [4]:
#For c = -3
c = -3 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP2(c,r)

dices = (1, 3, 3, 3, 3, 4, 4, 5, 6)
score = 0
used = set()

state = DiceState(dices, score, used)
mdp.explore(state)


print(mdp.opti[state], "value is:", mdp.value[state])


Continue with dice 4 value is: 1.4599163108402191


In [5]:
#Probability of obtaining a tile with 1 or more worms
from mdp2 import *
from dice_state import *

c = 0 #value if you lose the turn
r = [1]*100
mdp = MDP2(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print(p)

0.8930267371507402


In [6]:
#Probability of obtaining a tile with 2 or more worms
from mdp2 import *
from dice_state import *

c = 0 #value if you lose the turn
r = [0]*4+[1]*100
mdp = MDP2(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print(p)

0.6803307111795147


In [7]:
#Probability of obtaining a tile with 3 or more worms
from mdp2 import *
from dice_state import *

c = 0 #value if you lose the turn
r = [0]*8+[1]*100
mdp = MDP2(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print(p)

0.34629328714934327


In [8]:
#Probability of obtaining a tile with 4 worms
from mdp2 import *
from dice_state import *

c = 0 #value if you lose the turn
r = [0]*12+[1]*100
mdp = MDP2(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print(p)

0.08677521589317934


In [9]:
#Probability of getting exactly tile 24
from mdp2 import *
from dice_state import *

c = 0 #value if you lose the turn
r = [0,0,0,1] + [0]*100
mdp = MDP2(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print("Probability of getting exactly tile 24 is",p)


Probability of getting exactly tile 24 is 0.3750601883516776


In [10]:
#Probability of getting tile 27 or more
from mdp2 import *
from dice_state import *

c = 0 #value if you lose the turn
r = [0,0,0,0,0,0,1] + [1]*100
mdp = MDP2(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print("Probability of getting tile 27 of more is",p)

Probability of getting tile 27 of more is 0.5247594468429359


In [1]:
#Verification mdp
from dice_state import *
from mdp2 import *

c = 0 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP2(c,r)

dices = (1, 1, 2, 4, 4, 4, 5, 6)
score = 0
used = set()
state = DiceState(dices, score, used)

mdp.explore(state)

print(mdp.value[state], mdp.opti[state])

1.5035075126898982 Continue with dice 4


In [2]:
#Verification mdp
from dice_state import *
from mdp2 import *

c = 0 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP2(c,r)

dices = (1, 3, 4, 4, 6)
score = 9
used = {3}
state = DiceState(dices, score, used)

mdp.explore(state)

print(mdp.value[state], mdp.opti[state])
# state2 = (new_score, tuple(new_used), nb_dices-count)
#keeping the 2 four
state2 = (9+8,tuple({3,4}),3)
print("The expected value for keeping the 2 four is:",mdp.value_diceless[state2])

1.0007739864349952 Continue with dice 6
The expected value for keeping the 2 four is: 0.8577674897119344


In [3]:
#Verification mdp
from dice_state import *
from mdp2 import *

c = 0 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP2(c,r)

score = 23
used = {3,5,6}
nbdice = 3

state2 = score, tuple(used), nbdice

mdp.explore_diceless(state2)

print(f"{mdp.value_diceless[state2]} should stop : {mdp.should_stop[state2]}")

1.7256944444444444 should stop : False


In [4]:
#Verification mdp
from dice_state import *
from mdp2 import *

c = 0 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP2(c,r)

score = 21
used = {2,3,4,6}
nbdice = 2

state2 = score, tuple(used), nbdice

mdp.explore_diceless(state2)

print(f"{mdp.value_diceless[state2]} should stop : {mdp.should_stop[state2]}")

1 should stop : True


In [ ]:
from game_state import GameState, TileState
from dice_state import DiceState
from move import Move, MoveType
from player import GreedyPlayer
from run_game import *
from alpha_beta_player import AlphaBetaPlayer


p1 = AlphaBetaPlayer(0, 2)
n1 = f"AlphaBeta(0, 2)"
p2 = AlphaBetaPlayer(1,1)
n2 = f"AlphaBeta(1, 1)"

NUM_GAMES = 500

w, t = 0, 0
for i in range(NUM_GAMES):
    if i % 2 == 0:
        winner = run_game(p1, p2, debug=True)
    else:
        winner = run_game(p2, p1, debug=True)
    if winner == PlayerTurn.PLAYER_1:
        w+=1
    t += 1
    print(f"[{i}] {n1} wins with probabilty {w/t} over {n2}")



In [1]:
from alpha_beta_player import AlphaBetaPlayer
from game_state import GameState, TileState
from dice_state import DiceState
from move import Move, MoveType
from player import GreedyPlayer
from run_game import *

p1 = AlphaBetaPlayer(1,1)
p2 = AlphaBetaPlayer(1.5,0.5)

w = run_game(p1,p2)
print(w)

PlayerTurn.PLAYER_1


In [ ]:
from alpha_beta_player import AlphaBetaPlayer
from game_state import GameState, TileState
from dice_state import DiceState
from move import Move, MoveType
from player import GreedyPlayer
from run_game import *
from mdp import *

m = MDP(0, [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4])

state = DiceState((2, 4, 6, 6, 6), 15, {4})

m.explore(state)

print(m.opti[state])
for i in m.opti:
    print(i,m.opti[i])

In [ ]:
from ucb import *
from alpha_beta_player import AlphaBetaPlayer


p1 = AlphaBetaPlayer(1,1)

num_games = 10000
grid_size = 9

best_pair = simulate_games_ucb(p1, grid_size, num_games)
print(f"The best pair against (1,1) is: {best_pair}")

In [15]:
from reinforce import *
from constants import *
from feature_map import *


#doit être de la même taile que phi
w0 = np.array([1 for i in range(9)])
#w1 = [ 5.50374473, -1.36858401,  0.51578084,  3.28446589,  1.71115451,  5.64062541, -0.03022021,  1.78369999,  1.12533504,  1.12793183]
w1 = [ 9.9627381,  -3.108125,    1.28592067,  2.07558112,  2.4117237,   9.27304839, 1.07186631,  1.00073226,  1.07025064]
w2 = [ 0.31486198, -2.06873916,  0.97608623, 2.01753759,  2.80780986,  6.72828187, 1.25194883,  1.1615092,   0.51187523]

pr = Reinforce(w0,1,1,phi,0.95)


In [21]:
pr.train(2000,0.01)

step: 0
w: [1.00495422 1.         0.94657931 1.3237469  2.7055132  5.39390718
 1.09283325 0.79738258 0.70883394]
gradient norm: 0.3785323269704447
step: 100
w: [1.25247802 1.         0.94692609 1.32128017 2.60896973 5.48651276
 1.09230616 0.79874366 0.70918698]
gradient norm: 0.027142686652742186
step: 200
w: [0.99600493 1.         0.94702415 1.32137917 2.75085369 5.62299183
 1.09230647 0.79836191 0.70918812]
gradient norm: 0.08540710734464232
step: 300
w: [0.60717639 1.         0.94713396 1.32148694 2.82360384 5.65482276
 1.09230561 0.79767889 0.70918945]
gradient norm: 0.009403998613750958
step: 400
w: [0.6787399  1.         0.94722202 1.32155562 2.81418116 5.70568014
 1.09229655 0.79751074 0.70922188]
gradient norm: 0.27756228047195103
step: 500
w: [0.22948962 1.         0.94621547 1.3205517  2.90658224 5.61163073
 1.09229711 0.80132902 0.7092151 ]
gradient norm: 0.15633804556981595
step: 600
w: [0.50837259 1.         0.9470079  1.32134428 2.84838035 5.72007896
 1.0922972  0.7989633

In [22]:
pr.evaluate(1000)

0.389

In [4]:
print(pr.w)

[ 1.07817999 -2.61937355  0.85869885  1.12038423  2.77553292  6.31692917
  1.04826051  1.31944893  1.05957952]
